# DWD data preprocessing
This notebook will be used to test some of the gdal functions to prepare the DWD data to be used in SolEst later on.

In [8]:
import sys

In [9]:
sys.path.append('/Users/evandro/PycharmProjects/solest_data_handling/scripts')

In [18]:
from osgeo import gdal
import os
import config
import numpy as np
gdal.UseExceptions()

## 1 Extract header and data

In [19]:
# move back to main directory

In [20]:
os.chdir('/Users/evandro/PycharmProjects/solest_data_handling/')

In [21]:
sample_file = './data/dwd/global_irradiation/grids_germany_monthly_radiation_global_199401.asc'
parsed_file = './prepared_data/dwd/global_irradiation/grids_germany_monthly_radiation_global_199401.tif'

In [22]:
# 0. Get the actual data from the ascii file and parse the header
def parse_ascii_grid(file_path):
    header = {}
    data = []

    with open(file_path, 'r') as file:
        # Read header section
        line = file.readline()
        while line.strip() and not line.startswith("[ASCII-Raster-Format]"):
            if '=' in line:
                key, value = line.strip().split('=', 1)
                header[key] = value
            line = file.readline()

        for line in file:
            n_cells = len(line.split(" "))
            
            # ascii raster info
            if n_cells == 2:
                key, value = line.strip().split(' ', maxsplit=1)
                header[key] = value

            else:
                data.append(list(map(float, line.split(" "))))
    return header, data

## 2. Convert to GeoTIFF

In [23]:
def create_geotiff_from_data(data, header, output_filename):
    """ Create a GeoTIFF file from parsed data and header. """
    array = np.array(data)
    nrows, ncols = array.shape
    xllcorner = float(header['XLLCORNER'])
    yllcorner = float(header['YLLCORNER'])
    cellsize = float(header['CELLSIZE'])
    nodata = float(header['NODATA_VALUE'])

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(output_filename, ncols, nrows, 1, gdal.GDT_Float32, ["COMPRESS=LZW"])
    
    dataset.SetGeoTransform([xllcorner, cellsize, 0, yllcorner, 0, -cellsize])
    dataset.SetProjection('EPSG:31467')  # Set the projection

    band = dataset.GetRasterBand(1)
    band.SetNoDataValue(nodata)
    band.WriteArray(array)
    
    dataset.FlushCache()

In [24]:
header, data = parse_ascii_grid(sample_file)

In [25]:
create_geotiff_from_data(data, header, parsed_file)